<a href="https://colab.research.google.com/github/jeolin0525/github_repo/blob/master/PyDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dp # installing PyDP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 15.7 MB/s eta 0:00:00


In [2]:
import pydp as dp # by convention our package is to be imported as dp (dp for Differential Privacy!)
from pydp.algorithms.laplacian import BoundedSum, BoundedMean, Count, Max
import pandas as pd
import statistics 
import numpy as np
import matplotlib.pyplot as plt

Fetch all the required data!
The dataset used here contains 5000 records, and is stored across 5 files, each file containing 1000 records.

More specifically, the dataset contains details such as the first and last names, email addresses of customers and the amount they spent on purchasing goods, and the state in the US they're from.

Let's fetch all the records, read them into pandas DataFrames and take a look at the head of each of the DataFrames.

In [6]:
url1 = 'https://raw.githubusercontent.com/jeolin0525/github_repo/master/train.csv'
df1 = pd.read_csv(url1,sep=";", engine = "python")
df1.head()


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [12]:
url2 = 'https://raw.githubusercontent.com/jeolin0525/github_repo/master/test.csv.xls'
df2 = pd.read_csv(url2,sep=";", engine = "python")
df2.head()


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [9]:
df1.tail()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no
45210,37,entrepreneur,married,secondary,no,2971,no,no,cellular,17,nov,361,2,188,11,other,no


In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


In [14]:
combined_df_temp = [df1, df2]
original_dataset = pd.concat(combined_df_temp)
print(original_dataset.shape)

(49732, 17)


**Creating a Parallel Database**

Let us now create a parallel database that differs by only one record. 
We then inspect the heads of both DataFrames to verify that the first record has been removed.

In [19]:
redact_dataset = original_dataset.copy()
original_dataset.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


remove the first raw for redact_dataset (to check the differtial attack problem)

In [20]:
redact_dataset = redact_dataset[1:]
redact_dataset.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no


In [21]:
redact_dataset.education.unique()

array(['secondary', 'unknown', 'tertiary', 'primary'], dtype=object)

In [28]:
redact_dataset.job.unique()

array(['technician', 'entrepreneur', 'blue-collar', 'unknown',
       'management', 'retired', 'admin.', 'services', 'self-employed',
       'unemployed', 'housemaid', 'student'], dtype=object)

Is the amount of money we spend at our neighborhood store private or sensitive information? Well, it may not seem all that sensitive! But, what if the same information can be used to identify us?

In [30]:
original_dataset['balance'].max()

102127

In [29]:
original_dataset['balance'].min()

-8019

In [26]:
sum_original_dataset = round(sum(original_dataset['balance'].to_list()), 2)
print(sum_original_dataset,"<<origianl sum without dp>>")
sum_redact_dataset = round(sum(redact_dataset['balance'].to_list()), 2)
print(sum_redact_dataset,"<<react sum without dp>>")
sales_amount_Osbourne = round((sum_original_dataset - sum_redact_dataset), 2)
##assert sales_amount_Osbourne == original_dataset.iloc[0, 4]##
print(sales_amount_Osbourne, "=>1st balance")

68021518 <<origianl sum without dp>>
68019375 <<react sum without dp>>
2143 =>1st balance


**Differentially Private Sum**

Now, we illustrate how differentially private sum in place of simple sum can help in rendering membership inference attacks unsuccessful.

For the example above, let's assume that the customers should spend a minimum of 5\$ at the store and no more than 250$ for a particular purchase.

We then go ahead and compute differentially private sum on both original and the parallel dataset that differed by one record, as shown in the code snippets below.

In [31]:
dp_sum_original_dataset = BoundedSum(epsilon= 1.5, lower_bound = -8019, upper_bound = 102127, dtype ='float') 
dp_sum_og = dp_sum_original_dataset.quick_result(original_dataset['balance'].to_list())
dp_sum_og = round(dp_sum_og, 2)
print(dp_sum_og," <<dp_origin sum>>")

# Output dp_sum_og
dp_redact_dataset = BoundedSum(epsilon= 1.5, lower_bound =  -8019, upper_bound = 102127, dtype ='float')
dp_redact_dataset.add_entries(redact_dataset['balance'].to_list())
dp_sum_redact=round(dp_redact_dataset.result(), 2)
print(dp_sum_redact," <<dp_react sum>>")

# Output dp_sum_redact

68076208.06  <<dp_origin sum>>
68013082.85  <<dp_react sum>>
